# Import

In [1]:
import os
import re
import gc
import sys
from loguru import logger

import matplotlib.pyplot as plt 

from datetime import datetime

import numpy as np
import torch
import joblib

sys.path.append("../")

from HETSFileHelper import gatherCSV, readChannel, EIS_recal_ver02
from Outlier import OutlierDetection
from EISGPR import Interpolation

# Filesys

In [2]:
_path = "D:/Baihm/EISNN/Archive_New/"

for i in os.listdir(_path):
    
    print(i)

2025
2025.rar
4.zip
bad
removeEHD.ipynb


In [3]:
def SearchELE(rootPath, ele_pattern = re.compile(r"(.+?)_归档")):
    '''==================================================
        Search all electrode directories in the rootPath
        Parameter: 
            rootPath: current search path
            ele_pattern: electrode dir name patten
        Returen:
            ele_list: list of electrode directories
        ==================================================
    '''
    ele_list = []
    for i in os.listdir(rootPath):
        _path = os.path.join(rootPath, i)
        if os.path.isdir(_path):
            match_ele = ele_pattern.match(i)
            if match_ele:
                ele_list.append([_path, match_ele.group(1)])
            else:
                ele_list.extend(SearchELE(_path, ele_pattern))

    return ele_list


    

In [4]:
def setup_logger(log_dir="./LOG", log_filename="file.log", file_level="WARNING", console_level="WARNING"):
    # 创建目录
    os.makedirs(log_dir, exist_ok=True)
    log_fd = os.path.join(log_dir, log_filename)

    logger.remove()
    # 如果已有日志文件，重命名添加时间戳
    if os.path.exists(log_fd):
        name, ext = os.path.splitext(log_filename)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        archived_name = f"{name}_{timestamp}{ext}"
        archived_path = os.path.join(log_dir, archived_name)
        os.rename(log_fd, archived_path)

    # 添加终端输出
    logger.add(sys.stdout, level=console_level, enqueue=True)

    # 添加文件输出
    logger.add(log_fd, level=file_level, encoding="utf-8", enqueue=True)

    return logger

# Run

In [ ]:
setup_logger(log_dir="D:\Baihm\EISNN\LOG\Outlier_vitro_01")
# logger.remove()
# logger.add(sys.stdout, level="WARNING")
# logger.add("./LOG/file.log", rotation="10 MB", level="INFO")

<loguru.logger handlers=[(id=1, level=30, sink=stdout), (id=2, level=30, sink='D:\Baihm\EISNN\LOG\Outlier_vitro_01\file.log')]>

2025-05-09 14:02:03.134 | WARNING  | __main__:<module>:5 - Search in D:/Baihm/EISNN/Archive_New/ and find 187 electrodes
2025-05-09 14:02:07.547 | WARNING  | __main__:<module>:5 - ELE[1/187]: 	D:/Baihm/EISNN/Archive_New/2025\1月\02027452_归档
2025-05-09 14:02:07.548 | WARNING  | __main__:<module>:13 - FileAlreadyExistsWarning: 02027452 - 02027452_Outlier_Ver02.pt already exists.
2025-05-09 14:02:07.556 | WARNING  | __main__:<module>:5 - ELE[2/187]: 	D:/Baihm/EISNN/Archive_New/2025\1月\02027453_归档
2025-05-09 14:02:07.557 | WARNING  | __main__:<module>:13 - FileAlreadyExistsWarning: 02027453 - 02027453_Outlier_Ver02.pt already exists.
2025-05-09 14:02:07.564 | WARNING  | __main__:<module>:5 - ELE[3/187]: 	D:/Baihm/EISNN/Archive_New/2025\1月\06027742_归档
2025-05-09 14:02:07.566 | WARNING  | __main__:<module>:22 - IllegalInputError: 06027742 only has 3 samples.
2025-05-09 14:02:07.566 | WARNING  | __main__:<module>:5 - ELE[4/187]: 	D:/Baihm/EISNN/Archive_New/2025\1月\09047284_归档
2025-05-09 14:02:

In [6]:
rootPath = "D:/Baihm/EISNN/Archive_New/"
# rootPath = "D:/Baihm/EISNN/Dataset/"
ele_list = SearchELE(rootPath, ele_pattern=re.compile(r"(.+?)_归档"))
n_ele = len(ele_list)
logger.warning(f"Search in {rootPath} and find {n_ele:03d} electrodes")

## Each Electrode

In [7]:
DATASET_SUFFIX = "Outlier_Ver02"
SAVE_FLAG = False

In [8]:
freq_list   = np.linspace(0,5000-1,101,dtype=int, endpoint=True)

weirdModel  = joblib.load("./weirdSVMmodel.pkl")
openModel   = joblib.load("./openSVMmodel.pkl")
phz_calibration = np.loadtxt("./phz_Calib.txt")


In [ ]:


for i in range(n_ele):
# for i in range(0,1):
    elePath = ele_list[i][0]
    ele_id = ele_list[i][1]
    logger.warning(f"ELE[{i+1}/{n_ele}]: \t{elePath}")
    

    # Storage Preparing
    save_dir = f"{elePath}/{DATASET_SUFFIX}/"
    pt_file_name = f"{ele_id}_{DATASET_SUFFIX}.pt"
    os.makedirs(save_dir, exist_ok=True)
    if os.path.exists(os.path.join(save_dir, pt_file_name)):
        logger.warning(f"FileAlreadyExistsWarning: {ele_id} - {pt_file_name} already exists.")
        if SAVE_FLAG:
            continue


    # Load EIS data
    EISDict = gatherCSV(elePath)
    n_day   = len(EISDict)
    if n_day < 5:
        logger.warning(f"IllegalInputError: {ele_id} only has {n_day} samples.")
        continue
    try:
        x_day_full = [datetime.strptime(date, '%Y%m%d') for date in EISDict.keys()]
    except Exception as e:
        logger.error(f"IllegalDateError: {ele_id} has wrong date format. Please check the saving file. Error Code: {e}")
        continue

    _key    = next(iter(EISDict))
    n_ch    = len(EISDict[_key])
    
    if n_ch != 128:
        logger.warning(f"ChannelNumberWarning: {ele_id} only has {n_ch} channels.")
        continue
    


    # Iteration for each channel
    data_group = {}
    data_group['Channels']    = []
    ch_few_error   = []
    ch_open_error  = []
    ch_nan_error   = []
    ch_good        = []
    for j in range(n_ch):
    # for j in range(0,1):
        try:
            # logger.warning(f"ELE[{i+1}/{n_ele}] - ch[{j+1}/{n_ch}]")
            # logger.info(f"{EISDict}")
            chData_full = readChannel(j, EISDict)


            ## Outlier Detection
            eis_seq, eis_cluster, eis_anomaly, leaf_anomaly, seq_weird = OutlierDetection.OutlierDetection_Ver02(chData_full, mask_flag=False)
            seq_open, seq_short = OutlierDetection.OpenShortDetection(chData_full, mask_flag = False)

            ## Bad Electrode Detection
            # Too few normal samples 
            if np.shape(eis_seq)[0] < 5:
                ch_few_error.append(j)
                logger.warning(f"OutlierDetectionWarning: {ele_id} - CHID[{j}] only has {np.shape(eis_seq)[0]} valid samples.")
                continue
            
            # All normal samples is open / First normal sample is open
            if np.isin(eis_seq[0],seq_open):
                ch_open_error.append(j)
                logger.warning(f"OutlierDetectionWarning: {ele_id} - CHID[{j}] in Open state.")
                continue
                

            ## Calibration
            for k in range(np.shape(chData_full)[0]):
                ch_eis = EIS_recal_ver02(chData_full[k,:,:], phz_calibration)
                chData_full[k,:,:] = ch_eis

            chData = chData_full[:,:,freq_list]
            if np.isnan(chData).any():
                ch_nan_error.append(j)
                logger.warning(f"OutlierDetectionWarning: {ele_id} - CHID[{j}] chData Invalid")
                continue

            ch_good.append(j)


            ## Data Saving

            channel_group = {}
            channel_group['chData']         = chData
            channel_group['eis_seq']        = eis_seq
            channel_group['eis_cluster']    = eis_cluster
            channel_group['eis_anomaly']    = eis_anomaly
            channel_group['leaf_anomaly']   = leaf_anomaly
            channel_group['seq_weird']      = seq_weird
            channel_group['seq_open']       = seq_open
            channel_group['seq_short']      = seq_short

            data_group[j] = channel_group
            data_group['Channels'].append(j)




            ## Plot
            fig = plt.figure(figsize=(16, 9), constrained_layout=True)
            text_axis = OutlierDetection.OutlierDetectionPlot(fig, chData, eis_seq, eis_cluster, eis_anomaly, leaf_anomaly, seq_weird, seq_open, seq_short)
            font_properties = {
                'family': 'monospace',  
                'size': 14,             
                'weight': 'bold'        
            }

            text = f"EIE  : {ele_id}\nCHID : {j:03d}\nFrom : {x_day_full[0].strftime('%Y-%m-%d')}\nTo   : {x_day_full[-1].strftime('%Y-%m-%d')}"
            text_axis.text(0.2, 0.5, text, fontdict = font_properties, ha='left', va='center')

            # # Save Fig
            if SAVE_FLAG:
                fig_name = f"EISPure_{ele_id}_ch{j:03d}.png"
                
                os.makedirs(save_dir, exist_ok=True) 
                path = os.path.join(save_dir, fig_name)

                fig.savefig(path)
                plt.close(fig) 
            # else:
                # fig.show()

        except Exception as e:
            logger.warning(f"ELE[{i+1}/{n_ele}] - ch[{j+1}/{n_ch}] Run with error: {e}")
            continue
     
    # Storage Preparing
    pt_store = {}
    meta_group = {}
    meta_group["ele_id"]    = ele_id
    meta_group["elePath"]   = elePath
    meta_group["TimeSpan"]  = x_day_full
    meta_group["n_day"]     = n_day

    meta_group["n_ch"]          = n_ch
    meta_group["ch_few_error"]  = ch_few_error
    meta_group["ch_open_error"] = ch_open_error
    meta_group["ch_nan_error"]  = ch_nan_error
    meta_group["ch_good"]       = ch_good

    meta_group["Creater"]   = "Ming"
    meta_group['Date']      = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    pt_store['meta_group'] = meta_group
    pt_store['data_group'] = data_group
    if SAVE_FLAG:
        torch.save(pt_store, os.path.join(save_dir, pt_file_name))

    gc.collect()




# Load Test

In [10]:

if False:
    pt_name = "D:\Baihm\EISNN\Dataset\BatchRunTestDataset\\03018905_归档\Outlier_Ver02\\03018905_Outlier_Ver02.pt"
    loaded = torch.load(pt_name)

# Fix x_train

In [11]:


# MODEL_SUFFIX = "Matern12_Ver01"

# all_data_list = []

# for i in range(n_ele):
# # for i in range(3):
#     fd_pt = os.path.join(ele_list[i][0], MODEL_SUFFIX, f"{ele_list[i][1]}_{MODEL_SUFFIX}.pt")
#     if not os.path.exists(fd_pt):
#         # logger.warning(f"{fd_pt} does not exist")
#         continue
#     data_pt = torch.load(fd_pt, weights_only=False)
#     _meta_group = data_pt["meta_group"]
#     _data_group = data_pt["data_group"]

#     n_day       = _meta_group["n_day"]
#     n_ch        = _meta_group["n_ch"]
#     n_valid_ch  = len(_data_group["Channels"])